In [1]:
import random
import numpy as np
import pandas as pd
# from scipy.optimize import basinhopping
# from sklearn.metrics import mutual_info_score
from tqdm.notebook import tqdm_notebook
import time

In [2]:
import sys
sys.path.insert(0, "../ddf/")
import stainer as ST
import DirtyDF as ddf

# Create Demo Dataset

In [3]:
df = pd.read_csv("../../data/Telco-Customer-Churn.csv")

In [4]:
df = df.iloc[:99, :]

In [5]:
name_df = pd.read_csv("../../data/namelist.csv")

In [6]:
# pd.concat([cust_df, name_df], axis=1)

## Insert Names after customerID ##
df.insert(1, 'Name', name_df.loc[:, "Name"])
df

,customerID,Name,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Tristian Wunsch,Female,0,Yes,No,1,No,No phone service,DSL,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Rosamond Klocko,Male,0,No,No,34,Yes,No,DSL,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Georgianna Bahringer,Male,0,No,No,2,Yes,No,DSL,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Destiney Gutkowski,Male,0,No,No,45,No,No phone service,DSL,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Glenda Berge,Female,0,No,No,2,Yes,No,Fiber optic,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,9848-JQJTX,Terence Windler,Male,0,No,No,72,Yes,Yes,Fiber optic,...,Yes,No,Yes,Yes,Two year,Yes,Bank transfer (automatic),100.90,7459.05,No
95,8637-XJIVR,Ocie Pacocha,Female,0,No,No,12,Yes,Yes,Fiber optic,...,No,No,No,No,Month-to-month,Yes,Electronic check,78.95,927.35,Yes
96,9803-FTJCG,Zechariah Conn,Male,0,Yes,Yes,71,Yes,Yes,DSL,...,No,Yes,No,No,One year,Yes,Credit card (automatic),66.85,4748.70,No
97,0278-YXOOG,Lukas Gleichner,Male,0,No,No,5,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Mailed check,21.05,113.85,Yes


## Convert Types

In [8]:
# Need to manually convert to categorical types
df[df.columns.difference(['customerID', 'Name', 'SeniorCitizen', 'tenure', 'MonthlyCharges', 
                          'TotalCharges'])]=\
    df[df.columns.difference(['customerID', 'Name', 'SeniorCitizen', 'tenure', 'MonthlyCharges', 
                              'TotalCharges'])].astype('category')

In [9]:
df.dtypes

customerID            object
Name                  object
gender              category
SeniorCitizen          int64
Partner             category
Dependents          category
tenure                 int64
PhoneService        category
MultipleLines       category
InternetService     category
OnlineSecurity      category
OnlineBackup        category
DeviceProtection    category
TechSupport         category
StreamingTV         category
StreamingMovies     category
Contract            category
PaperlessBilling    category
PaymentMethod       category
MonthlyCharges       float64
TotalCharges         float64
Churn               category
dtype: object

# DDF Demo

In [11]:
# Create DDF object
telco_ddf = ddf.DirtyDF(df, seed=2101)

In [12]:
# Check for the column types
print(telco_ddf.cat_cols)
print(telco_ddf.num_cols)
print(telco_ddf.dt_cols)

[2, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 21]
[3, 6, 19, 20]
[]


## Adding Stainers

In [14]:
for idx, name in enumerate(df.columns):
    print(f"{idx} : {name}")

0 : customerID
1 : Name
2 : gender
3 : SeniorCitizen
4 : Partner
5 : Dependents
6 : tenure
7 : PhoneService
8 : MultipleLines
9 : InternetService
10 : OnlineSecurity
11 : OnlineBackup
12 : DeviceProtection
13 : TechSupport
14 : StreamingTV
15 : StreamingMovies
16 : Contract
17 : PaperlessBilling
18 : PaymentMethod
19 : MonthlyCharges
20 : TotalCharges
21 : Churn


## Stainers used for trial

In order to track what the true name should be, we need an unaltered __unique__ identifier column. 

In this case each `customerID` is unique and hence serves as a link to the true name (present in a dataset on our own side)

1. Row Duplication
2. Mutual Information (on `Contract` & `Churn`)
3. Inflection (on `PaymentMethod` & `Contract`)
4. Name Mangling (on `Name`)
5. Nullify Stainer (on `TotalCharges`)

__NOTE :__ 

When setting the run order for the stainers, stainers that perform manipulation of columns (eg. Mutual Information / Name Mangling) should be called before stainers that could interfere with their operations. 

Alternatively, could just have one stainer operate on one column in order to avoid conflicts. 

In [21]:
# Create DDF object
telco_ddf = ddf.DirtyDF(df, seed=2101)

## Instantiate Stainers ##
# Duplication Stainer
dupli_stainer=ST.RowDuplicateStainer(deg=0.2, max_rep=3) 
# Mutual Information
mut_stainer=ST.CatCorrStainer(col_idx=[16, 21], max_n=50000, min_inf=0.4)
# Name Mangling
mangler_stainer = ST.NameManglerStainer(col_idx=[1])
# Inflection stainer. Need to specify column idxs in list
inflection_stainer=ST.InflectionStainer(num_format=3, col_idx=[16, 18])
# Nullify Stainer
null_stainer=ST.NullifyStainer(deg=0.25, col_idx=[20])

In [22]:
## Add Stainers ##
telco_ddf=telco_ddf.add_stainers([dupli_stainer, mut_stainer, mangler_stainer,
                                  inflection_stainer, null_stainer])

In [23]:
## Run Stainers ##
telco_ddf=telco_ddf.run_all_stainers()

In [24]:
telco_ddf.print_history()

1. Add Duplicates 
 Added Duplicate Rows for 19 rows. 
  Each duplicated row should appear a maximum of 3 times. 
  Rows added: 29 
 Time taken: 0.021883726119995117 

2. CatCorr 
 Old Mutual Information was 0.2407920518041773, new Mutual Information is 0.4033728704360404 
 Time taken: 0.03774213790893555 

3. NameMangler 
 Name Mangling completed using 5 iterations of mangling on each FullName 
 Time taken: 0.007306814193725586 

4. Inflection 
 Categorical inflections on:
{'Contract': {'Month-to-month': ['month_to_month', 'Month-to-month', 'Month To Month'], 'One year': ['one year', 'One Year', 'One year'], 'Two year': ['Two Year', 'two year', 'Two year']}, 'PaymentMethod': {'Electronic check': ['Electronic check'], 'Mailed check': ['Mailed check'], 'Bank transfer (automatic)': ['Bank transfer (automatic)'], 'Credit card (automatic)': ['Credit card (automatic)']}} 
 Time taken: 0.004011869430541992 

5. Nullify 
 Replaced 32 values to become empty in specificed rows/cols. 
 Time take

## Demo Result

In [25]:
result = telco_ddf.get_df()
result.head()

,customerID,Name,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,"Dr.Tristian. Wu-nsch, PHD",Female,0,Yes,No,1,No,No phone service,DSL,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Mrs.Rosamond-- Klo-cko,Male,0,No,No,34,Yes,No,DSL,...,Yes,No,No,No,Month-to-month,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Mr.Georgianna.-- Bahringer,Male,0,No,No,2,Yes,No,DSL,...,No,No,No,No,Month To Month,Yes,Mailed check,53.85,NaN,No
3,7795-CFOCW,Mr. Dr. Destiney- Gu-tko-wski,Male,0,No,No,45,No,No phone service,DSL,...,Yes,Yes,No,No,Month To Month,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Mr. Mr. Glenda-.Berge,Female,0,No,No,2,Yes,No,Fiber optic,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,NaN,No


## Comparison with True Name

In [32]:
comparison = pd.merge(result[['customerID', "Name"]], df[['customerID', "Name"]], on='customerID', how = "left",
                     suffixes = ("_Mangled", "_Original"))

In [34]:
comparison.sort_values(by='customerID')

,customerID,Name_Mangled,Name_Original
104,0191-ZHSKZ,"Dave. Sha-na-han, PHD, PHD",Dave Shanahan
126,0278-YXOOG,"Dr.Lukas Gle-i-chner, PHD",Lukas Gleichner
19,0280-XJGEX,Dr.Tommie.Co-llier,Tommie Collier
112,0318-ZOPWS,"Ce-le-stino- Ho-we, CFA",Celestino Howe
84,0434-CSFON,Roberto.-He-gma-nn,Roberto Hegmann
...,...,...,...
58,9867-JCZSP,Mrs. Mr. Lorine.-Leannon,Lorine Leannon
111,9919-YLNNG,"Emi-l Je-rde, CFA, M.D.",Emil Jerde
110,9919-YLNNG,Dr. Mr. Emil-Je-rde,Emil Jerde
109,9919-YLNNG,Emil---Je-rde,Emil Jerde
